In [1]:
"""
README: YOLO Detection Failure Analysis Script

This script analyzes detection "failure types" by comparing YOLO-format ground truth
and prediction label files. It reports three error types per image:
- partial: predicted box overlaps only partially with ground truth (IoU > 0.1 but < 0.5)
- occluded: a ground truth object overlaps with another unpredicted ground truth box (IoU > 0.3)
- superimposed: more than one prediction matches to a single ground truth object

All results are exported as a JSON file for further analysis.

How to use:
- Place your YOLO-format ground truth and prediction .txt files in two separate directories.
- Edit the `gt_dir` and `pred_dir` paths at the bottom of this script.
- Run the script. Results will be written to `failure_analysis.json`.

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import os
import cv2
import json
from pathlib import Path

def iou(boxA, boxB):
    xa, ya, wa, ha = boxA
    xb, yb, wb, hb = boxB
    xa1, ya1, xa2, ya2 = xa - wa/2, ya - ha/2, xa + wa/2, ya + ha/2
    xb1, yb1, xb2, yb2 = xb - wb/2, yb - hb/2, xb + wb/2, yb + hb/2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    areaA = (xa2 - xa1) * (ya2 - ya1)
    areaB = (xb2 - xb1) * (yb2 - yb1)
    return inter_area / (areaA + areaB - inter_area + 1e-6)

def load_yolo_txt(path):
    with open(path) as f:
        lines = [l.strip().split() for l in f if l.strip()]
    return [(int(l[0]), list(map(float, l[1:5]))) for l in lines]

def analyze_failures(gt_dir, pred_dir, output_json="failure_analysis.json"):
    results = []
    gt_dir = Path(gt_dir)
    pred_dir = Path(pred_dir)

    for gt_file in sorted(gt_dir.glob("*.txt")):
        image_id = gt_file.stem
        pred_file = pred_dir / gt_file.name
        if not pred_file.exists():
            continue

        gt_data = load_yolo_txt(gt_file)
        pred_data = load_yolo_txt(pred_file)

        gt_boxes = [box for cls, box in gt_data]
        pred_boxes = [box for cls, box in pred_data]

        failure_types = {"partial": 0, "occluded": 0, "superimposed": 0}
        matched_gt = set()
        pred_match_counts = [0] * len(pred_boxes)

        for i, gt in enumerate(gt_boxes):
            best_iou = 0
            best_j = -1
            for j, pred in enumerate(pred_boxes):
                iou_val = iou(gt, pred)
                if iou_val > best_iou:
                    best_iou = iou_val
                    best_j = j

            if best_iou >= 0.5:
                matched_gt.add(i)
                pred_match_counts[best_j] += 1
            elif best_iou > 0.1:
                failure_types["partial"] += 1

        # Occlusion: very close two GT boxes, one not detected
        for i, box1 in enumerate(gt_boxes):
            for j, box2 in enumerate(gt_boxes):
                if i != j and i not in matched_gt:
                    if iou(box1, box2) > 0.3:
                        failure_types["occluded"] += 1
                        break

        # Superimposed: more than one prediction per GT
        for count in pred_match_counts:
            if count > 1:
                failure_types["superimposed"] += 1

        results.append({
            "image": image_id,
            "partial": failure_types["partial"],
            "occluded": failure_types["occluded"],
            "superimposed": failure_types["superimposed"]
        })

    with open(output_json, "w") as f:
        json.dump(results, f, indent=2)
    print(f"Results saved: {output_json}")

# === USAGE ===
gt_dir = "PATH_TO_GROUND_TRUTH_TXT"
pred_dir = "PATH_TO_PREDICTION_TXT"
analyze_failures(gt_dir, pred_dir)


✅ Sonuçlar kaydedildi: failure_analysis.json
